In [1]:
import pandas as pd
import numpy as np
import pystan
import warnings
from bokeh.io import output_notebook
from bokeh.plotting import figure, show
from bokeh.models import Span

output_notebook()
warnings.filterwarnings("ignore")

Loading BokehJS ...

## Prepare poll data

In [2]:
df = pd.read_csv('data/president_general_polls_2016.csv')
polls = df[['pollster', 'grade', 'state', 'createddate', 'samplesize', 'rawpoll_trump', 'rawpoll_clinton']]
polls.createddate = pd.to_datetime(polls.createddate)

pc = ['AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'DC', 'FL', 'GA', 'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 
      'KY', 'LA', 'ME', 'MD', 'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY', 'NC', 
      'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY']

nb_voters = [2074338,301694,2323579,1078548,13202158,2596173,1560640,413921,294254,8538264,3919355,437159,666290,
             5279752,2663368,1589951,1182771,1815843,2014548,724758,2734062,3184196,4780701,2950780,1285584,
             2757323,491966,804245,1016664,718700,3683638,786522,7128852,4542488,325564,5632423,1334872,1820507,
             5742040,446049,1981516,368270,2478870,7993851,1028786,301793,3888186,3172939,670438,3068434,250701]

nb_electors = [9, 3, 11, 6, 55, 9, 7, 3, 3, 29, 16, 4, 4, 20, 11, 6, 6, 8, 8, 4, 10, 11, 16, 10, 6, 10, 3, 5, 6, 
               4, 14, 5, 29, 15, 3, 18, 7, 7, 20, 4, 9, 3, 11, 38, 6, 3, 13, 12, 5, 10, 3]

states = sorted(list(set(df.state.values)))
to_remove = ['U.S.', 'Nebraska CD-1', 'Nebraska CD-2', 'Nebraska CD-3', 'Maine CD-1', 'Maine CD-2']
for s in to_remove: states.remove(s)
polls = polls[polls.state.isin(states)]

polls.dropna(inplace=True)
grades = ['A+', 'A', 'A-', 'B+', 'B', 'B-', 'C+', 'C', 'C-',  'D']
polls.grade = polls.grade.apply(lambda x: 10 - grades.index(x))

polls.samplesize = polls.samplesize.astype(np.int32)
polls['trump'] = (polls.samplesize*polls.rawpoll_trump/100).astype(np.int32)
polls['clinton'] = (polls.samplesize*polls.rawpoll_clinton/100).astype(np.int32)
polls['independent'] = (polls.samplesize - (polls.clinton + polls.trump))

polls.drop_duplicates(inplace=True)

aux = polls[polls.createddate >= pd.to_datetime('2016/11/01')]
polls_b1_lw = aux.sort('grade', ascending=False).groupby('state', as_index=False).nth(list(range(1)))

aux = polls[polls.createddate > pd.to_datetime('2016/10/07')]
polls_b3_lm = aux.sort('grade', ascending=False).groupby('state', as_index=False).nth(list(range(3)))

aux = polls.copy()
polls_b3 = aux.sort('grade', ascending=False).groupby('state', as_index=False).nth(list(range(3)))

polls_all_lw = polls[polls.createddate >= pd.to_datetime('2016/11/01')]

print(len(polls_b1_lw), len(polls_b3_lm), len(polls_b3), len(polls_all_lw))

polls = {
    'best_last_week': {
        'votes': polls_b1_lw[['trump', 'clinton', 'independent']].values, 
        'states': np.array([states.index(s) for s in polls_b1_lw.state.values])
    },
    'best3_last_month': {
        'votes': polls_b3_lm[['trump', 'clinton', 'independent']].values, 
        'states': np.array([states.index(s) for s in polls_b3_lm.state.values])
    },
    'best3': {
        'votes': polls_b3[['trump', 'clinton', 'independent']].values, 
        'states': np.array([states.index(s) for s in polls_b3.state.values])
    },
    'all_last_week': {
        'votes': polls_all_lw[['trump', 'clinton', 'independent']].values, 
        'states': np.array([states.index(s) for s in polls_all_lw.state.values])
    }
}

(51, 153, 153, 808)


# Stan model

In [ ]:
code = '''
data {
    int nb_polls; // Number of polls
    int nb_states; // Number of states (51 because of D.C.)
    int nb_candidates; // Number of candidates (3: Trump, Clinton, Ind.)
    int polls_states[nb_polls]; // Poll -> state map
    int votes[nb_polls, nb_candidates]; // Polled votes for each candidate
    int nb_voters[nb_states]; // Number of voters for forecasting
    real bias; // Polling bias
}
parameters {
    simplex[nb_candidates] theta[nb_states]; //1 - Trump, 2 - Clinton, 3 - Ind.
    vector[nb_candidates] alpha;
}
model {
    for(c in 1:nb_candidates)
        alpha[c] ~ uniform(0, 1000); // Weakly informative hyperprior

    for(s in 1:nb_states)
        theta[s] ~ dirichlet(alpha); // Dirichlet prior per state

    for(p in 1:nb_polls) // Multinomial observations (polled values)
        votes[p] ~ multinomial(theta[polls_states[p]]);
}
generated quantities {
    int votes_pred[nb_states, nb_candidates]; // Predicted number of votes on election day
    real epsilon[nb_states]; // Bias random variable
    simplex[nb_candidates] theta_bias[nb_states]; // Biased voting intentions

    // The deltas below are used to ensure that the biased thetas form a valid simplex
    real delta_t[nb_states];
    real delta_h[nb_states];
    real delta[nb_states];

    for(s in 1:nb_states) {
        if(bias == 0.0)
            epsilon[s] <- 0.0;
        else
            epsilon[s] <- uniform_rng(0, bias); // Bias value for this state

        // We must ensure that theta will remain a valid probability simplex,
        // so we limit delta in a way theta will never be below 0 or above 1
        delta_t[s] <- fabs(theta[s][1] - fmax(0.0, fmin(1.0, theta[s][1] + epsilon[s])));
        delta_h[s] <- fabs(theta[s][2] - fmin(1.0, fmax(0.0, theta[s][2] - epsilon[s])));
        delta[s] <- fmin(delta_t[s], delta_h[s]);

        theta_bias[s][1] <- theta[s][1] + delta[s];
        theta_bias[s][2] <- theta[s][2] - delta[s];
        theta_bias[s][3] <- theta[s][3];

        votes_pred[s] <- multinomial_rng(theta_bias[s], nb_voters[s]);
    }
}
'''

model = pystan.StanModel(model_code=code)

# Forecast with different biases

In [ ]:
winners = []
biases = np.linspace(0.0, 0.05, 20)
for b in biases:
    data = {
        'nb_states': 51,
        'nb_candidates': 3,
        'nb_polls': len(polls['best_last_week']['votes']),
        'polls_states': polls['best_last_week']['states']+1,
        'votes': polls['best_last_week']['votes'],
        'nb_voters': nb_voters,
        'bias': b
    }
    
    fit = model.sampling(data=data, iter=10000, thin=2, chains=4, seed=0)
    trace = fit.extract()
    winner = []
    for election_sample in trace['votes_pred']:
        cnts = np.zeros(3)
        for s, tci in enumerate(election_sample):
            cnts[tci.argmax()] += nb_electors[s]
        if cnts.max() >= 270:
            winner.append(cnts.argmax())
        else:
            winner.append(-1)
    winners.append(np.histogram(winner, bins=3)[0]/float(len(winner)))
winners = np.array(winners)

In [ ]:
p = figure(title="Polling bias influence on forecast", plot_height=400, plot_width=550, x_range=(0, 5), y_range=(0, 100))
p.line(biases*100, winners[:, 1]*100, line_width=3, color='red', legend='Trump')
p.line(biases*100, winners[:, 2]*100, line_width=3, color='blue', legend='Clinton')
p.line(biases*100, winners[:, 0]*100, line_width=3, color='green', legend='Neither')
p.title.align = 'center'

intersec = Span(location=2.004, dimension='height', line_color='black', line_dash='dashed', line_width=1)
p.add_layout(intersec)
p.legend.location = 'right_center'
p.xaxis[0].axis_label = 'Trump favorable bias (%)'
p.yaxis[0].axis_label = 'Probability of winning (%)'
show(p)